In [1]:
import numpy as np
import tensorflow as tf
from PIL import Image
!pip install pycocotools --user

2022-03-14 23:49:46.010828: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-14 23:49:46.010880: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [18]:
model = tf.keras.models.load_model('classmodel')

In [19]:
#probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

In [27]:
img = Image.open('300.3.jpg')
img_resize = img.copy()

In [28]:
img_resized = img_resize.resize((900,900))

In [29]:
img_test = (np.expand_dims(img_resized,0))

In [30]:
predictions_single = model.predict(img_test)

print(predictions_single)

[[-23.16564    -2.5418847  25.866259 ]]


In [31]:
predictions_single

array([[-23.16564  ,  -2.5418847,  25.866259 ]], dtype=float32)

In [9]:
predict = np.argmax(predictions_single[0])
print(predict)

2


In [10]:
predictions_single[0][2]

1.0

In [11]:
img.size

(851, 817)

In [12]:
img_resized.size

(900, 900)